In [2]:
import cv2
from ultralytics import YOLO
from collections import defaultdict
model = YOLO("yolo11l")  

In [3]:
class_list = model.names
class_list

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [ ]:
import cv2

# Load a frame from your video
cap = cv2.VideoCapture('test_videos/5.mp4')
ret, frame = cap.read()
cap.release()

points = []

def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # left mouse click
        points.append((x, y))
        print(f"Point selected: ({x}, {y})")
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Select Points", frame)

cv2.imshow("Select Points", frame)
cv2.setMouseCallback("Select Points", mouse_callback)

cv2.waitKey(0)
cv2.destroyAllWindows()

print("Final Points:", points)


Point selected: (276, 219)
Point selected: (354, 215)
Point selected: (447, 240)
Point selected: (556, 296)


In [1]:
import cv2
import math
from ultralytics import YOLO
from collections import defaultdict

model = YOLO("yolo11l.pt")
class_list = model.names

cap = cv2.VideoCapture("test_videos/5.mp4")
if not cap.isOpened():
    print("❌ Could not open video. Check path and file.")
    exit()

# ---------------- Define Lines ----------------
line1_pt1 = (276, 219)
line1_pt2 = (354, 215)
line2_pt1 = (447, 240)
line2_pt2 = (556, 296)

# ---------------- Helpers ----------------
def point_to_segment_distance(pt, a, b):
    px, py = pt
    x1, y1 = a
    x2, y2 = b
    dx = x2 - x1
    dy = y2 - y1
    if dx == 0 and dy == 0:
        return math.hypot(px - x1, py - y1)
    t = ((px - x1) * dx + (py - y1) * dy) / (dx*dx + dy*dy)
    t = max(0.0, min(1.0, t))
    proj_x = x1 + t*dx
    proj_y = y1 + t*dy
    return math.hypot(px - proj_x, py - proj_y)

def is_on_line_segment(pt, line_pt1, line_pt2, thresh=8):
    return point_to_segment_distance(pt, line_pt1, line_pt2) <= thresh

# ---------------- Tracking State ----------------
cross_status = {}   # track_id -> {"line1": False, "line2": False}
counts_line1 = defaultdict(int)
counts_line2 = defaultdict(int)

# Priority flag: None until first crossing happens
priority = None   # can be "line1" or "line2"

print("Press 'q' or ESC to exit, or click the window close button.")
def draw_transparent_box(frame, x1, y1, x2, y2, color, alpha=0.3):
    """Draw semi-transparent filled box with border"""
    overlay = frame.copy()
    cv2.rectangle(overlay, (x1, y1), (x2, y2), color, -1)  # filled
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)  # border


# ---------------- Main Loop ----------------
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True, classes=[1,2,3,5,6,7])

    if results and len(results) > 0 and getattr(results[0], "boxes", None) is not None:
        boxes = results[0].boxes.xyxy.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        class_indices = results[0].boxes.cls.int().cpu().tolist()

        # Draw reference lines
        cv2.line(frame, line1_pt1, line1_pt2, (0, 0, 255), 2)   # line1 = red
        cv2.line(frame, line2_pt1, line2_pt2, (0, 255, 0), 2)   # line2 = green

        for box, tid, cls_idx in zip(boxes, track_ids, class_indices):
            x1, y1, x2, y2 = map(int, box)
            cx, cy = (x1 + x2)//2, (y1 + y2)//2
            class_name = class_list[cls_idx]

            if tid not in cross_status:
                cross_status[tid] = {"line1": False, "line2": False}

            # Detect crossing for Line 1
            if not cross_status[tid]["line1"] and is_on_line_segment((cx,cy), line1_pt1, line1_pt2):
                cross_status[tid]["line1"] = True
                counts_line1[class_name] += 1
                if priority is None:
                    priority = "line1"   # Line1 gets priority

            # Detect crossing for Line 2
            if not cross_status[tid]["line2"] and is_on_line_segment((cx,cy), line2_pt1, line2_pt2):
                cross_status[tid]["line2"] = True
                counts_line2[class_name] += 1
                if priority is None:
                    priority = "line2"   # Line2 gets priority

            # Decide color based on priority
            color = (0, 255, 255)  # yellow (not crossed yet)
            if priority == "line1":
                if cross_status[tid]["line1"]:
                    draw_transparent_box(frame, x1, y1, x2, y2, (0, 255, 0))   # allowed
                elif cross_status[tid]["line2"]:
                    draw_transparent_box(frame, x1, y1, x2, y2, (0, 0, 255)) 
                else:
                    draw_transparent_box(frame, x1, y1, x2, y2, (0, 255, 255))
            elif priority == "line2":
                if cross_status[tid]["line2"]:
                    draw_transparent_box(frame, x1, y1, x2, y2, (0, 255, 0))  # allowed
                elif cross_status[tid]["line1"]:
                    draw_transparent_box(frame, x1, y1, x2, y2, (0, 0, 255))  # blocked
                else:
                    draw_transparent_box(frame, x1, y1, x2, y2, (0, 255, 255))

            # Draw object
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"ID:{tid} {class_name}", (x1, y1-8),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.circle(frame, (cx, cy), 3, color, -1)

        # Show counts
        oy = 30
        cv2.putText(frame, f"Priority: {priority if priority else 'None'}", (10, oy-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
        cv2.putText(frame, "Line1 counts:", (10, oy+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
        oy += 40
        for k, v in counts_line1.items():
            cv2.putText(frame, f"{k}: {v}", (10, oy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
            oy += 20
        oy += 10
        cv2.putText(frame, "Line2 counts:", (10, oy), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
        oy += 30
        for k, v in counts_line2.items():
            cv2.putText(frame, f"{k}: {v}", (10, oy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
            oy += 20

    cv2.imshow("YOLO Priority Lane Tracking", frame)

    # Exit handling (Mac safe)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == 27:
        break
    if cv2.getWindowProperty("YOLO Priority Lane Tracking", cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


Press 'q' or ESC to exit, or click the window close button.



/Users/sayanbiswas/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


0: 384x640 8 cars, 220.8ms
Speed: 1.8ms preprocess, 220.8ms inference, 8.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 189.7ms
Speed: 1.3ms preprocess, 189.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 190.3ms
Speed: 1.6ms preprocess, 190.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 189.0ms
Speed: 0.9ms preprocess, 189.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 185.4ms
Speed: 1.7ms preprocess, 185.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 186.6ms
Speed: 0.8ms preprocess, 186.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 184.4ms
Speed: 0.8ms preprocess, 184.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 188.9ms
Speed: 0.7ms preprocess, 188.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384

-1